In [45]:
# IMPORT LIBRARIES AND MODULES
import paramiko
import logging
from paramiko_expect import SSHClientInteraction
from paramiko.ssh_exception import AuthenticationException, SSHException, NoValidConnectionsError
import json
import xlsxwriter
import re



In [46]:
# IMPORT THE CLASS MODULES
# The DataAnalysis class reads command outputs from a text file, parses and analyzes server replication statuses, and logs errors or successful tests.

from data_analysis import DataAnalysis

In [29]:
# INPUT FILES - SERVER DETAILS & COMMANDS LIST
server_file = 'uc_server.json'
command_file = 'commandList.txt'

In [47]:
# READ THE SERVER DETAILS FROM JSON FILE AND FILTER OUT CUCM SERVERS AND STORE IN A LIST
server_list = []
cm_server = []

with open(server_file) as f:
    server_list = json.load(f)
    print(server_list)
    for srv in server_list:
        ip = srv['ip']
        os_uname = srv['os_uname']
        os_pass = srv['os_pass']
        type = srv['type']
        print(ip, os_uname, os_pass, type)
        if type == 'CUCM':
            cm_server.append(srv)

print("Server List: ", server_list)
print("CM Server List: ", cm_server)

[{'ip': '198.18.133.3', 'os_uname': 'administrator', 'os_pass': 'dCloud123!', 'type': 'CUCM'}]
198.18.133.3 administrator dCloud123! CUCM
Server List:  [{'ip': '198.18.133.3', 'os_uname': 'administrator', 'os_pass': 'dCloud123!', 'type': 'CUCM'}]
CM Server List:  [{'ip': '198.18.133.3', 'os_uname': 'administrator', 'os_pass': 'dCloud123!', 'type': 'CUCM'}]


In [48]:
# READ THE COMMANDS FROM THE COMMAND FILE AND STORE IN A LIST
commands = []
with open(command_file, 'r') as file:
    commands = [line.strip() for line in file if line.strip()]
    print(commands)

['utils dbreplication runtimestate']


In [49]:
# USING PARAMIKO TO CONNECT TO SERVERS

def connect(ip, username, password):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    try:
        ssh.connect(ip, username=username, password=password)
        interact = SSHClientInteraction(ssh, timeout=60, display=True)
        logging.info(f"Successfully connected to {ip}")
        return ssh, interact
    except (AuthenticationException, SSHException, NoValidConnectionsError) as e:
        logging.error(f"Failed to connect to {ip}: {str(e)}")
        raise e

In [50]:
# CLOSE THE CONNECTION AFTER EXECUTING COMMANDS
def close(ssh):
    if ssh:
        try:
            ssh.close()
            print(f"Connection closed")
        except Exception as e:
            print(f"Failed to close connection: {str(e)}")
            raise e

In [51]:
# SAMPLE CODE TO CONNECT TO SERVERS
for cm in cm_server:
    ip = cm['ip']
    os_uname = cm['os_uname']
    os_pass = cm['os_pass']
    try:
        ssh, interact = connect(ip, os_uname, os_pass)
        print("Connected to ", ip)
        print(ssh, interact)
        close(ssh)
    except Exception as e:
        print(e)
        continue


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.8)
INFO:paramiko.transport:Authentication (password) successful!
INFO:root:Successfully connected to 198.18.133.3


Connected to  198.18.133.3
<paramiko.client.SSHClient object at 0x0000025112BE8290> <paramiko_expect.SSHClientInteraction object at 0x0000025112C8FBC0>
Connection closed


In [52]:
# CONNECT TO SERVERS, NOW EXECUTE COMMANDS

def execute_command(interact, command):
    try:
        interact.send(command)
        interact.expect('admin:', timeout=700)
        output = interact.current_output_clean
        print(output)
        logging.info(f"Command executed successfully: {command}")
        return output
    except Exception as e:
        logging.error(f"Failed to execute command {command}: {str(e)}")
        raise e

In [53]:
# SAMPLE CODE TO EXECUTE COMMANDS AFTER CONNECTING TO SERVERS
output = "HEALTH CHECK OUTPUT \n\n"
for cm in cm_server:
    ip = cm['ip']
    os_uname = cm['os_uname']
    os_pass = cm['os_pass']
    ssh, interact = connect(ip, os_uname, os_pass)
    print("Connected to ", ip)
    print(ssh, interact)
    for command in commands:
        print(command)
        output = output + execute_command(interact, command)
        print(output)
    close(ssh)

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.8)
INFO:paramiko.transport:Authentication (password) successful!
INFO:root:Successfully connected to 198.18.133.3


Connected to  198.18.133.3
<paramiko.client.SSHClient object at 0x0000025112BF7E60> <paramiko_expect.SSHClientInteraction object at 0x00000251124DB590>
utils dbreplication runtimestate
utils dbreplication runtimestate
Command Line Interface is starting up, please wait ...

   Welcome to the Platform Command Line Interface

VMware Installation:
	2 vCPU: Intel(R) Xeon(R) CPU E7- 2830  @ 2.13GHz
	Disk 1: 150GB, Partitions aligned
	8192 Mbytes RAM

admin:

INFO:root:Command executed successfully: utils dbreplication runtimestate


utils dbreplication runtimestate
Command Line Interface is starting up, please wait ...

   Welcome to the Platform Command Line Interface

VMware Installation:
	2 vCPU: Intel(R) Xeon(R) CPU E7- 2830  @ 2.13GHz
	Disk 1: 150GB, Partitions aligned
	8192 Mbytes RAM


HEALTH CHECK OUTPUT 

utils dbreplication runtimestate
Command Line Interface is starting up, please wait ...

   Welcome to the Platform Command Line Interface

VMware Installation:
	2 vCPU: Intel(R) Xeon(R) CPU E7- 2830  @ 2.13GHz
	Disk 1: 150GB, Partitions aligned
	8192 Mbytes RAM


Connection closed


In [54]:
# SAVE EXECUTED COMMAND OUTPUT TO A FILE
def save_output(ip, commands, output,count):
    ip_file_name = ip.replace('.', '_')
    output_file = f"{ip_file_name}.txt"
    with open(output_file, 'w') as out:
        out.write(output)
    print(f"Output saved to {output_file}")

    workbook = xlsxwriter.Workbook('output.xlsx')
    worksheet = workbook.add_worksheet()
    worksheet.write('A' + str(count), ip)
    worksheet.write('B' + str(count), commands)
    worksheet.write('C' + str(count), output)
    # save the file
    workbook.close()


In [55]:
# SAMPLE CODE TO CONNECT TO SERVERS, EXECUTE COMMANDS, SAVE OUTPUT AND CLOSE CONNECTION
count = 0
for cm in cm_server:
    ip = cm['ip']
    os_uname = cm['os_uname']
    os_pass = cm['os_pass']
    try:
        ssh, interact = connect(ip, os_uname, os_pass)
        print("Connected to ", ip)
        print(ssh, interact)
        for command in commands:
            output = execute_command(interact, command)
            save_output(ip, command ,output, count)
            count = count + 1
        close(ssh)
    except Exception as e:
        print(e)
        continue

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.8)
INFO:paramiko.transport:Authentication (password) successful!
INFO:root:Successfully connected to 198.18.133.3


Connected to  198.18.133.3
<paramiko.client.SSHClient object at 0x0000025112BF7770> <paramiko_expect.SSHClientInteraction object at 0x0000025112BF7FB0>
utils dbreplication runtimestate
Command Line Interface is starting up, please wait ...

   Welcome to the Platform Command Line Interface

VMware Installation:
	2 vCPU: Intel(R) Xeon(R) CPU E7- 2830  @ 2.13GHz
	Disk 1: 150GB, Partitions aligned
	8192 Mbytes RAM

admin:utils dbreplication runtimestate


Server Time: Wed Feb 12 17:13:56 CST 2025

Cluster Replication State: BROADCAST SYNC ended at: 2025-02-10-23-14
     Sync Result: SYNC COMPLETED on 753 tables out of 753
     Sync Status: All Tables are in sync
     Use CLI to see detail: 'file view activelog cm/trace/dbl/20250210_231106_dbl_repl_output_Broadcast.log'

DB Version: ccm15_0_1_11900_23

Repltimeout set to: 300s
PROCESS option set to: 1


Cluster Detailed View from cucm1 (3 Servers):

                                           PING      DB/RPC/   REPL.    Replication    REPL

INFO:root:Command executed successfully: utils dbreplication runtimestate


utils dbreplication runtimestate
Command Line Interface is starting up, please wait ...

   Welcome to the Platform Command Line Interface

VMware Installation:
	2 vCPU: Intel(R) Xeon(R) CPU E7- 2830  @ 2.13GHz
	Disk 1: 150GB, Partitions aligned
	8192 Mbytes RAM

admin:utils dbreplication runtimestate


Server Time: Wed Feb 12 17:13:56 CST 2025

Cluster Replication State: BROADCAST SYNC ended at: 2025-02-10-23-14
     Sync Result: SYNC COMPLETED on 753 tables out of 753
     Sync Status: All Tables are in sync
     Use CLI to see detail: 'file view activelog cm/trace/dbl/20250210_231106_dbl_repl_output_Broadcast.log'

DB Version: ccm15_0_1_11900_23

Repltimeout set to: 300s
PROCESS option set to: 1


Cluster Detailed View from cucm1 (3 Servers):

                                           PING      DB/RPC/   REPL.    Replication    REPLICATION SETUP
SERVER-NAME              IP ADDRESS        (msec)    DbMon?    QUEUE    Group ID       (RTMT) & Details
-----------              ---------

In [56]:
# ANALYZE THE OUTPUT FILE

file_path = "10_29_194_204.txt"

analysis = DataAnalysis(file_path)
analysis.read_cmd_output()
results = analysis.analyze_dbreplication_output()
print(results)

Analyzing output for command(s): ['utils dbreplication runtimestate']
Error: DB/RPC/mon for ucmsub2 is Y/Y/N, should be Y/Y/Y.
Error: Replication setup for ucmsub2 is 'Out Of Sync', should be 'Setup Completed'.
Error: Replication setup for ucmsub1 is 'Out Of Sync', should be 'Setup Completed'.
